# Banco de dados Solo Criado: Take 1

Vou fazer primeiro no notebook para ir testando, depois passo para o arquivo .py

## Roadmap

* Coletar listagem do site da prefeitura
* Transformar pdf da listagem em xmls por este site
* Transformar xmls em cvs
* Carregar cvs de listagem para o banco de dados
  * Extração;
  * Exploração;
  * Limpeza;
  * Agregação;
  * Visualização;
  * Storytelling.

## Primeira parte: Coleta e transformação dos dados

from [here](https://www.edureka.co/blog/web-scraping-with-python/): <br>
Libraries used for Web Scraping  <br>
As we know, Python is has various applications and there are different libraries for different purposes. In our further demonstration, we will be using the following libraries: <br>
<br>
**Selenium**:  Selenium is a web testing library. It is used to automate browser activities. <br>
**BeautifulSoup**: Beautiful Soup is a Python package for parsing HTML and XML documents. It creates parse trees that is helpful to extract the data easily.<br>
Pandas: Pandas is a library used for data manipulation and analysis. It is used to extract the data and store it in the desired format. 

Eu tenho que baixar com requests somente a parte que me interessa (div id="conteudo" class="outros") <br>
Ou usar o bs4 pra retirar somente essa parte do todo

O código abaixo recebe e cria um arquivo html com os dados da página de listagens.

In [ ]:
#import requests
#
#url = 'http://www2.portoalegre.rs.gov.br/edificapoa/default.php?p_secao=1445'
#r = requests.get(url)
#
#open('estoques.html', 'wb').write(r.content)

O código abaixo (em andamento) serve para limpar o html e eventualmente retirar os links das listagens em pdf

In [ ]:
#from bs4 import BeautifulSoup
#
#with open('estoques.html') as fp:
#    soup = BeautifulSoup(fp, 'html.parser')
#
#conteudo = soup.find(id="conteudo")
#conteudo.find_all('strong')
#conteudo.find_all('a')
#for link in conteudo.find_all('a'):
#    print(link.get('href'))

Ao longo do tempo o formato de cada entrada de listagem mudou, então o ideal seria criar uma lista com as listagens já existentes e ir adicionando com base na ultima listagem. O problema é se mudarem o formato de divulgação de novo. acho que vou focar somente na parte de banco de dados que á mais útil e confiável, por enquanto.

## Segunda parte: a criação do banco de dados

### 1. Extração

imports

In [ ]:
import pandas as pd
import os
#import numpy as np
import functools as ft

lista de arquivos

In [ ]:
print(os.listdir("./dados/ListagemSoloCriado/XLSX"))

join_sheet

In [ ]:
def join_sheet(file):
    """Receives a excel file path and
    returns a dataframe with his sheets
    appended. Also fix third column name."""

    sheet_list = pd.ExcelFile(file).sheet_names
    df_list = []
    for sheet in sheet_list:
        df_list.append(pd.read_excel(io=file, sheet_name=sheet, header=1))
    
    listagem = pd.concat(objs=df_list, ignore_index=True)
    
    colunas = listagem.columns.to_list()
    colunas[-1] = 'estq_max_' + file[-7:-5]
    listagem.columns = colunas

    return listagem


mzueuqrt

In [ ]:
def mzueuqrt(dataframe: list):
    """Receives a dataframe, adds
    the MZUEUQRT column in the [-1]
    position, and makes it the index."""

    new_columns = list(dataframe.columns)
    new_columns.insert(-1, 'MZUEUQRT')
    dataframe = dataframe.reindex(columns=new_columns)
    #Agora vem o calculo da columa MZUEUQRT
    
    dataframe['MZUEUQRT'] = (dataframe['MZ']*1000000) + (dataframe['UEU']*1000) + (dataframe['QRT'])

    return dataframe

db_manager

In [ ]:
def db_manager(path=any, database=[]):
    """Receives a path to Excel files and 
    a list containing dataframes, checks if 
    there is any file that is not in the database, 
    adds the missing files to database if there is any 
    and returns the updated database."""

    files = os.listdir(path)
    files.sort()
    
    if len(database) == len(files): #Não há nenhum dado novo a ser adicionado
        print("O banco de dados já está atualizado.")
        return database
    elif len(database) > len(files): #Há algum problema, pois o BD possui mais entradas do que a pasta de arquivos
        print("O banco de dados possui mais entradas do que a pasta de arquivos")
        return database
    elif len(database) < len(files): #Existem novas listagens que não foram adicionadas ao BD
        print("O banco de dados está desatualizado")
        for i in range(len(database), len(files)):
            listagem = mzueuqrt(join_sheet(path + "/" + files[i]))
            listagem = listagem.set_index('MZUEUQRT')
            database.append(listagem)
            print("Adicionando listagem", (i+1))

    return database
    

Testando a função db_manager

In [ ]:
db_estq = db_manager("./dados/ListagemSoloCriado/XLSX")


In [ ]:
#estq_max_01 = db_estq[0]
#estq_max_02 = db_estq[1]
#estq_max_03 = db_estq[2]
#estq_max_04 = db_estq[3]
#estq_max_05 = db_estq[4]
#estq_max_06 = db_estq[5]
#estq_max_07 = db_estq[6]
#estq_max_08 = db_estq[7]
#estq_max_09 = db_estq[8]
#estq_max_10 = db_estq[9]
#estq_max_11 = db_estq[10]
#estq_max_12 = db_estq[11]
#estq_max_13 = db_estq[12]
#estq_max_14 = db_estq[13]
#estq_max_15 = db_estq[14]
#estq_max_16 = db_estq[15]

## Terceira parte: Manipulação do Banco de Dados

### Agregação

estoque_maximo

In [ ]:
def estoque_maximo(database):
    """Receives a database containing 
    all dataframes, and aggregates them."""

    estq_max = database[0]

    for i in range(1, len(database)):
        if i < 9: #Somente para não ter erro com a coluna da direita        \/
            estq_max = pd.merge(left=estq_max, right=database[i][('estq_max_0'+str(i+1))], how='outer', on='MZUEUQRT')
        else:
            estq_max = pd.merge(left=estq_max, right=database[i][('estq_max_'+str(i+1))], how='outer', on='MZUEUQRT')
    
    return estq_max

testando a função estoque_maximo

In [ ]:
estq_max_num = estoque_maximo(db_estq)
estq_max_num

In [ ]:
#estq_max_num.index[estq_max_num['estq_max_01'] = = 'AE'].tolist()

In [ ]:
#type(estq_max_num['estq_max_01'][1004005])
estq_max_num['estq_max_01']

Função que separa as áreas especiais

In [ ]:
def retira_ae(dataframe):
    colunas = list(dataframe.columns)

    for i in range(3, len(colunas)):
        area_especial = list(dataframe.index[dataframe[colunas[i]] == 'AE'])
        dataframe.drop(labels=area_especial, axis=0, inplace=True)
    
    return dataframe


In [ ]:
def retorna_ae(dataframe):
    colunas = list(dataframe.columns)

    for i in range(3, len(colunas)):
        area_especial = list(dataframe.index[dataframe[colunas[i]] != 'AE'])
        dataframe.drop(labels=area_especial, axis=0, inplace=True)
    
    return dataframe

In [ ]:
#area_especial = list(estq_max_num.index[estq_max_num['estq_max_01'] == 'AE'])
area_especial = retira_ae(estq_max_num)
area_especial.head(5)

In [ ]:
teste = pd.DataFrame({'a': [1, 2, 3], 'b': [2, 3, 4], 'c': [3, 4, 5]})

colunas = list(teste.columns)
teste[colunas[2]] = teste[colunas[0]] - teste[colunas[1]]
teste

In [ ]:
rem = teste.index[teste['a'] == 2].tolist()
teste.drop(labels=rem, axis=0) #sucesso

Agora uma função que cria o dataframe de uso de solo criado

In [ ]:
def uso_estoque(dataframe):
     dataframe = retira_ae(dataframe)
     colunas = list(dataframe.columns) 

     df2 = dataframe
     for i in range(4, len(colunas)):
          df2[colunas[i]] = (dataframe[colunas[3]] - dataframe[colunas[i]])
          colunas[i] = 'uso_estq_' + colunas[i][-2:]
     
     colunas[3] = 'uso_estq_' + colunas[3][-2:]
     df2.columns = colunas
     
     return dataframe

In [ ]:
uso_estq = uso_estoque(estq_max_num)
#uso_estq
estq_max_num